In [3]:
import sympy
from sympy import sympify, lambdify, symbols, integrate, Interval, Symbol, I, S, oo

QUALIFIED_FUNCTIONS = {sympy.Abs, sympy.sin, sympy.cos, sympy.sqrt, sympy.log, sympy.tan, sympy.atan}

# A test case
expr=sympify('log(sin(x1)) - I*x1')
print(type(expr))
print(expr.free_symbols)
print("test:", Symbol('x1') in expr.free_symbols)
print("Complex?", expr.has(I))
print("is_Function:", expr.is_Function)
print("is Expr:", isinstance(expr, sympy.Expr))
print(expr.atoms(sympy.Function))
print("Is function qualified:", type(list(expr.atoms(sympy.Function))[0]) in QUALIFIED_FUNCTIONS)


# Parse an expression. 
# Only returns parsed expression if it is qualified. Otherwise returns None.
# An expr is qualified if it meets all following conditions:
#  1. Has designated number of symbols
#  2. Does not have unknown functions
#  3. Has no imaginary number
# After parsed, replace the symbols to x1, x2, etc. Replace symbol "I" to 1.
def parse_expression(expr_str, min_num_symbols, max_num_symbols):
    try:
        expr=sympify(expr_str)
        if expr.has(I):
            return None
        funcs = expr.atoms(sympy.Function)
        for func in funcs:
            if type(func) not in QUALIFIED_FUNCTIONS:
                return None
        variables = list(expr.free_symbols)
        #print("variables:", variables)
        if len(variables) < min_num_symbols or len(variables) > max_num_symbols:
            return None
        qualified_names = {'x1', 'x2', 'x3', 'x4', 'x5', 'x6'}
        replacements = {Symbol("I"):1}
        for var in variables:
            if var.name in qualified_names:
                continue
            for i in range(1, len(variables)+1):
                s = Symbol('x' + str(i))
                if s not in variables and s not in replacements.values():
                    replacements[var] = s
                    break
        return expr.subs(replacements)
    except:
        return None

parse_expression('Abs(cos(y)+sin(x3))+sqrt(log(x1))', 1, 10)

<class 'sympy.core.add.Add'>
{x1}
test: True
Complex? True
is_Function: False
is Expr: True
{sin(x1), log(sin(x1))}
Is function qualified: True


sqrt(log(x1)) + Abs(sin(x3) + cos(x2))

In [4]:
# Parse all combination expressions

import json
from collections import defaultdict

expr_freq = defaultdict(int)

fin = open('/home/mcwave/code/automath/calculus/datasets/intermediate_algebra.json', 'r')
lines = fin.readlines()
for line in lines:
    d = json.loads(line)
    if "eqt" in d:
        eqt = d["eqt"]
        if len(eqt) >= 100:
            continue
        #print(eqt, type(eqt))
        expr = parse_expression(eqt, 2, 6)
        if expr is not None:
            expr_freq[expr] = expr_freq[expr] + 1

fin.close()
expr_sorted = sorted(expr_freq.items(), key=lambda item: item[1], reverse=True)
expr_sorted

[(x1*x2, 143),
 (x1 + x2, 134),
 (x1 + x2 + x3, 114),
 (x1 + x2 + x3 + x4, 47),
 (-x1 + x2, 42),
 (x1**2 + x2**2, 35),
 (x1 - x2, 30),
 (x1**2 + x2**2 + x3**2, 26),
 (x1*x2*x3, 26),
 (x1 + x2*x3**2 + x3*x4, 25),
 (x1/x2, 24),
 (x1*x2 + x1*x3 + x2*x3, 18),
 (Abs(x1 - x2), 17),
 (x1*x3 + x2 + x3**2, 16),
 (x2/x1, 14),
 (x1 + x2 + x3 + x4 + x5, 13),
 (x1 + 1/x2, 12),
 (x2 + 1/x1, 10),
 (x1 + x2**2 + x2*x3, 9),
 (x1 + x2*x3**2 + x3**3 + x3*x4, 9),
 (x1**2 + x2**2 + x3**2 + x4**2, 8),
 (x1*x3 + x2, 8),
 (x1**4 + x2**4 + x3**4, 7),
 (4*x1 + 3*x2, 7),
 (x2**x1, 7),
 (x1**3 + x2**3 + x3**3, 7),
 (1/x2 + 1/x1, 7),
 (x1**2 + 2*x1*x2 + x2**2, 6),
 (x1**x2, 6),
 (x1**2 + x2**2 + 2*x3**2, 5),
 (x1 + x2*x5**3 + x3*x5 + x4*x5**2 + 4*x5**4, 5),
 (Abs(x1*x4 + x2*x3), 5),
 (x1*x2**2 + 6, 5),
 (E*x1*x2*x3*x4, 5),
 (-2*x1 + x2, 5),
 (x1*x2 + 6*x2**3 + 19*x2**2 + 35, 5),
 (-6*x1**2 + x2**2, 5),
 (6*x1 + 12*x2 + 4, 5),
 (x1 + 4*x2 + 12*x3, 5),
 (6*x1 + 14*x2 + 6, 5),
 (x1*x2 + x2**3 - 3*x2**2 + 27, 5),
 (x1

In [ ]:
# Parse all single symbol expressions

import json
from collections import defaultdict

expr_freq = defaultdict(int)

fin = open('/home/mcwave/code/automath/calculus/datasets/intermediate_algebra.json', 'r')
lines = fin.readlines()
for line in lines:
    d = json.loads(line)
    if "eqt" in d:
        eqt = d["eqt"]
        if len(eqt) >= 100:
            continue
        #print(eqt, type(eqt))
        expr = parse_expression(eqt, 2, 6)
        if expr is not None:
            expr_freq[expr] = expr_freq[expr] + 1

fin.close()
expr_sorted = sorted(expr_freq.items(), key=lambda item: item[1], reverse=True)
expr_sorted

In [51]:
import numpy as np
from sympy.calculus.util import continuous_domain, function_range

DEFAULT_LEFT = -2
DEFAULT_RIGHT = 2

f1 = sympify("t**2")
f2 = sympify("log(t)")
x1 = Symbol('x1')
x2 = Symbol('x2')
combined_expr = sympify("x1+x2")
combined_expr = combined_expr.subs({x1:f1, x2:f2})
t = Symbol('t')
dom1 = continuous_domain(f1, t, S.Reals)
dom2 = continuous_domain(f2, t, S.Reals)
dom = dom1.intersect(dom2)
intv = dom.intersect(Interval(DEFAULT_LEFT, DEFAULT_RIGHT))
print("interval:", intv)

result = integrate(combined_expr, (t, intv.left, intv.right))
print(result.evalf())
result

interval: Interval.Lopen(0, 2)
2.05296102778656


2/3 + 2*log(2)

In [21]:
import sympy
from sympy import sympify, lambdify, symbols, integrate, Interval, Symbol, I, S, oo

x = Symbol('x')
root = sympify("(-b+sqrt(b**2-4*a*c))/2/a")
expr = sympify("a*x**2+b*x+c")
sympy.simplify(expr.subs({x:root}))

# #p,q,r = symbols(['p', 'q', 'r'])
# p = sympify('-b/(3*a)')
# q = sympify('p**3 + (b*c-3*a*d)/(6*a**2)')
# r = sympify('c/(3*a)')
# root = sympify('(q + (q**2 + (r-p**2)**3)**(1/2))**(1/3) + (q - (q**2 + (r-p**2)**3)**(1/2))*(1/3) + p')
# #root = root.subs({p:p1, q:q1, r:r1})
# expr = sympify('a*x**3 + b*x**2 + c*x + d')
# sympy.simplify(expr.subs({x:root}))

0

In [12]:
a, b = symbols(['a', 'b'])

In [14]:
type(a)

sympy.core.symbol.Symbol